In [3]:
import os
import math
from numpy import * 
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

### 若要測試單一個，請以以下方式引入單一資料集
<span id="singleRunHead"></span>

In [1]:
# dir_data = '../datasets/'
# DATASET_CSV_NAME = '2_Landsat.csv'
# DATASET_NAME = 'Landsat'
# dir_train_test_data = f'../train_test_datasets/{DATASET_NAME}'
# train_data_loc = os.path.join(dir_data, DATASET_CSV_NAME)
# print('Path of read in data: %s' % (train_data_loc))
# data = pd.read_csv(train_data_loc)
# data.head()

In [2]:
# len(data.columns)

In [22]:
# train_1_data_loc = os.path.join(dir_train_test_data, 'train_1.csv')
# train_1 = pd.read_csv(train_1_data_loc)
# train_2_data_loc = os.path.join(dir_train_test_data, 'train_2.csv')
# train_2 = pd.read_csv(train_2_data_loc)
# train_3_data_loc = os.path.join(dir_train_test_data, 'train_3.csv')
# train_3 = pd.read_csv(train_3_data_loc)
# train_4_data_loc = os.path.join(dir_train_test_data, 'train_4.csv')
# train_4 = pd.read_csv(train_4_data_loc)
# train_5_data_loc = os.path.join(dir_train_test_data, 'train_5.csv')
# train_5 = pd.read_csv(train_5_data_loc)
# test_1_data_loc = os.path.join(dir_train_test_data, 'test_1.csv')
# test_1 = pd.read_csv(test_1_data_loc)
# test_2_data_loc = os.path.join(dir_train_test_data, 'test_2.csv')
# test_2 = pd.read_csv(test_2_data_loc)
# test_3_data_loc = os.path.join(dir_train_test_data, 'test_3.csv')
# test_3 = pd.read_csv(test_3_data_loc)
# test_4_data_loc = os.path.join(dir_train_test_data, 'test_4.csv')
# test_4 = pd.read_csv(test_4_data_loc)
# test_5_data_loc = os.path.join(dir_train_test_data, 'test_5.csv')
# test_5 = pd.read_csv(test_5_data_loc)
# del train_1['numbers']
# del train_2['numbers']
# del train_3['numbers']
# del train_4['numbers']
# del train_5['numbers']
# del test_1['numbers']
# del test_2['numbers']
# del test_3['numbers']
# del test_4['numbers']
# del test_5['numbers']
# training_Data = []
# training_Data.append(train_1)
# training_Data.append(train_2)
# training_Data.append(train_3)
# training_Data.append(train_4)
# training_Data.append(train_5)
# testing_Data = []
# testing_Data.append(test_1)
# testing_Data.append(test_2)
# testing_Data.append(test_3)
# testing_Data.append(test_4)
# testing_Data.append(test_5)
# training_Data[0]

### Library

In [2]:
from sklearn.feature_selection import mutual_info_classif, VarianceThreshold
from sklearn.feature_selection import chi2, f_classif
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import SelectKBest
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc, f1_score, confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, MinMaxScaler
import matplotlib.pyplot as plt
from ReliefF import ReliefF
from sklearn.linear_model import LassoCV, RidgeCV, Ridge
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

### Classifier

In [29]:
# svm = SVC(kernel='rbf', probability=True, decision_function_shape='ovr', random_state=42) ## classifier
knn = KNeighborsClassifier()
# svm = LinearSVC(multi_class='crammer_singer')

### Datasets tags

In [27]:
DATASETS = [
    'Landsat',
    'soybean',
    'waveform',
    'Splice',
    'urban_land_cover','SCADI', 
    'lung_discrete',
    'CNAE-9',
    'oh15','lung',
    'TOX_171',
    'orlraws10P',
    'CLL_SUB_111',
#     'CARCINOM',
#     'GLIOMA',
#     'AllBooks_baseline_DTM_Labelled'
]
    
DATASETS_CSV = [
    '2_Landsat',
    '3_soybean', 
    '4_waveform',
    '5_Splice',
    '6_urban_land_cover',
    '7_SCADI',
    '9_lung_discrete',
    '11_CNAE-9',
    '12_oh15',
    '13_lung',
    '14_TOX_171',
    '15_orlraws10P',
    '16_CLL_SUB_111',
#     'CARCINOM',
#     'GLIOMA',
#     'AllBooks_baseline_DTM_Labelled'
]

### Decomposition

#### OVA

In [5]:
'''
args:
    train_array: 訓練的資料，每個trainData要呼叫一次以執行 decomposition
return:
    data_temp: 進行decomposition後的資料集，會在最後加上 class_ova 的標籤
'''
def decomposition(train_array):
    class_ova = [] ## ova label 佔存，每次迴圈插入進當次的 ova dataframe
    data_temp = [] ## ova dataframe container，分別會有「是 i 與非 i 的 label」
    for i in range(data['class'].value_counts().size):
        data_temp.append(train_array.copy())
        class_ova = []
        for j in range(train_array["class"].size):
            if(data_temp[i]["class"][j] == i+1):
                class_ova.append(1)
            elif(data_temp[i]["class"][j] != i+1):
                class_ova.append(2)
        data_temp[i][str("class_ova_")+str(i+1)] = class_ova
    return data_temp

#### ovo

In [6]:
from itertools import combinations
def combination(n, k):
    if k > n:
        return 0
    else:
        combinations_list = list(combinations(range(1, n+1), k))
        
        for i in range(len(combinations_list)):
            combinations_list[i] = [eval(str(j)) for j in combinations_list[i]]
        
        return combinations_list
# result= combination(9, 2)
# print(result)
# print(len(result))

In [7]:
def decomposition_ovo(train_array):   
    c = combination(CLASS_AMOUNT, 2)
    ovo_container = []
    for i in range(len(c)):
        
        class_ova = []
        t = train_array.copy()
        tt1 = t.loc[train_array['class'] == c[i][0]]
        tt1 = tt1.reset_index(drop=True).astype('int')
        tt2 = t.loc[train_array['class'] == c[i][1]]
        tt2 = tt2.reset_index(drop=True).astype('int')
        for j in range(tt1['class'].size):
            if (tt1['class'][j] != 1 or tt1['class'][j] == 1):
                class_ova.append(1)
        for j in range(tt2['class'].size):
            if(tt2['class'][j] != 1):
                class_ova.append(2)
        tt3 = pd.concat([tt1, tt2]).reset_index(drop=True)
        tt3['class_ovo'] = class_ova
#     tt3
        ovo_container.append(tt3)
    return ovo_container, len(c)

### Feature Selection Algos

In [8]:
def fs_embedded_ova(cols_container, data_temp, embedded_algo):
    #     print("-------fs-------")
#     data['class'].value_counts().size
    for i in range(data['class'].value_counts().size):
        data_ova = data_temp[i].copy()
#         data_forXGBoost = data_temp[i].copy()
            
        mms = pd.DataFrame(data=data_ova.drop(['class', str('class_ova_')+str(i+1)], axis=1))
        data_ova_mxs = MinMaxScaler().fit_transform(mms)

        data_ova_mxs = pd.DataFrame(data=data_ova_mxs, columns=data_ova.drop(['class', str('class_ova_')+str(i+1)], axis=1).columns)
        data_ova_mxs['class'] = data_ova['class']
        data_ova_mxs[str('class_ova_')+str(i+1)] = data_ova[str('class_ova_')+str(i+1)]
        data_forXGBoost = data_ova[str('class_ova_')+str(i+1)].copy()

        del data_ova_mxs['class']  
        data_ova_no_label = data_ova_mxs.copy()
        del data_ova_no_label[str('class_ova_')+str(i+1)]
#     try:
        if(embedded_algo == 'Lasso'):
            reg = LassoCV(random_state=2)
            reg.fit(data_ova_no_label, data_ova[str('class_ova_')+str(i+1)])
        elif(embedded_algo == 'Ridge'):
            reg = RidgeCV()
            reg.fit(data_ova_no_label, data_ova[str('class_ova_')+str(i+1)])                
        elif(embedded_algo == 'RandomForest'):
            reg = RandomForestClassifier(n_estimators=100, random_state=2)
            reg.fit(data_ova_no_label, data_ova[str('class_ova_')+str(i+1)])                
        elif(embedded_algo == 'XGBoost'):
            reg = XGBClassifier()
            for i in range(len(data_forXGBoost)):
                data_forXGBoost[i] = int(data_forXGBoost[i])-1
            reg.fit(data_ova_no_label, data_forXGBoost)


        data_ova_no_label = data_ova_no_label.loc[:, data_ova_no_label.columns != 'class']
        if(embedded_algo == 'RandomForest' or embedded_algo == 'XGBoost'):
            importances = reg.feature_importances_
            selected_columns = [feature for feature, weight in zip (data_ova_no_label.columns, importances) if weight != 0] 
            cols_container.append(selected_columns)
        elif(embedded_algo == 'Lasso' or embedded_algo == 'Ridge'):
            selected_columns = [feature for feature, weight in zip (data_ova_no_label.columns, reg.coef_) if weight != 0] 
            cols_container.append(selected_columns)
#     except:
#         print("something wrong")
#         cols_container = []
    return cols_container

In [9]:
def fs_embedded_ovo(cols_container, data_temp, looptimes, embedded_algo):
    for i in range(looptimes):
        data_ovo = data_temp[i].copy()
#         data_forXGBoost = data_temp[i].copy()
            
        mms = pd.DataFrame(data=data_ovo.drop(['class', 'class_ovo'], axis=1))
        data_ovo_mxs = MinMaxScaler().fit(mms).transform(mms) 
            
        data_ovo_mxs = pd.DataFrame(data=data_ovo_mxs, columns=data_ovo.drop(['class', 'class_ovo'], axis=1).columns)

        data_ovo_mxs['class'] = data_ovo['class']
        data_ovo_mxs['class_ovo'] = data_ovo['class_ovo']
        data_forXGBoost = data_ovo['class_ovo'].copy()

        del data_ovo_mxs['class']
        data_ovo_no_label = data_ovo_mxs.copy()
        del data_ovo_no_label['class_ovo']
        
#     try:
        if(embedded_algo == 'Lasso'):
            reg = LassoCV(random_state=2)
            reg.fit(data_ovo_no_label, data_ovo['class_ovo'])
        elif(embedded_algo == 'Ridge'):
            reg = RidgeCV()
            reg.fit(data_ovo_no_label, data_ovo['class_ovo'])                
        elif(embedded_algo == 'RandomForest'):
            reg = RandomForestClassifier(n_estimators=100, random_state=2)
            reg.fit(data_ovo_no_label, data_ovo['class_ovo'])                
        elif(embedded_algo == 'XGBoost'):
            reg = XGBClassifier()
            for i in range(len(data_forXGBoost)):
                data_forXGBoost[i] = int(data_forXGBoost[i])-1
#                 print(data_forXGBoost[i])
            reg.fit(data_ovo_no_label, data_forXGBoost)


        data_ovo_no_label = data_ovo_no_label.loc[:, data_ovo_no_label.columns != 'class']
        if(embedded_algo == 'RandomForest' or embedded_algo == 'XGBoost'):
            importances = reg.feature_importances_
            selected_columns = [feature for feature, weight in zip (data_ovo_no_label.columns, importances) if weight != 0] 
            cols_container.append(selected_columns)
        elif(embedded_algo == 'Lasso' or embedded_algo == 'Ridge'):
            selected_columns = [feature for feature, weight in zip (data_ovo_no_label.columns, reg.coef_) if weight != 0] 
            cols_container.append(selected_columns)
#     except:
#         print("something wrong")
#         cols_container = []
    return cols_container

### Union and Intersection

In [10]:
'''
args:
    cols_container: fs 後的 f 集合，將每個 decomposition 後的 f 集合進行聯集
return:
    list(C): 把聯集後的結果回傳
'''
def Union(cols_container):
#     fs - Union
#     print("-------fs-Union-------")
    union_temp = cols_container.copy()
#     print(len(cols_container))
    A = set(union_temp[0]) # 1
    B = set(union_temp[1]) # 2
    C = A.union(B)
    for i in range(2, len(union_temp)):
        A = set(union_temp[i])
        C = A.union(C)
#     print(C)
#     print('length of union set' + str(len(C)))
#     print(f'length of union set: {len(C)}')
    return list(C), len(C)

In [11]:
'''
args:
    cols_container: fs 後的 f 集合，將每個 decomposition 後的 f 集合進行交集
return:
    list(C): 把交集後的結果回傳
'''
def Intersection(cols_container):
    ## fs - intersection
#     print("-------fs-intersection-------")
    intersection_temp = cols_container.copy()
    A = set(intersection_temp[0]) # 1
    B = set(intersection_temp[1]) # 2
    C = A.intersection(B)
    for i in range(2, len(intersection_temp)):
        A = set(intersection_temp[i])
        C = A.intersection(C)
#     print(C)
#     print(f'length of intersection set: {len(C)}')
    return list(C), len(C)

### train model

In [12]:
def train_test_clean(train_data, test_data, choosedFeature_AfterAction):
    temp = train_data.copy()
    del temp['class']
    temp_selected= temp[choosedFeature_AfterAction]
    x_train = temp_selected.sort_index(axis=1)
#     print(x_train) #80筆資料

    #把測試集手動取出與訓練集相同維度的所有資料
    temp_test = test_data.copy() #x_test
    del temp_test['class']
    temp_test_selected = temp_test[choosedFeature_AfterAction]
    x_test = temp_test_selected.sort_index(axis=1)
#     print(x_test)

    y_train = train_data['class']
    y_test = test_data['class']
    
    return x_train, x_test, y_train, y_test
def train_model(x_train, x_test, y_train, y_test, classifier):
    ## train model
#     print(x_train)
    classifier.fit(x_train, y_train)
    predicted = classifier.predict(x_test) # 預測的答案
    
#     print(predicted)
    '''-------------'''
#     predict_proba = classifier.predict_proba(x_test)
#     predict_proba = OneHotEncoder().fit_transform(predicted.reshape(-1,1)).toarray()
#     predict_proba = list(predict_proba)

#     not_predicted = []
#     for i in range(len(arange(1,7))):
#         if(i+1 not in predicted):
#             not_predicted.append(i+1)
# #     print(not_predicted)
#     if(not_predicted):
#         for i in range(len(not_predicted)):
#             for j in range(len(predict_proba)):
#                 predict_proba[j] = insert(predict_proba[j], not_predicted[i]-1, 0.)
#     print(predict_proba)
    '''-------------'''

    '''-------------'''
    lb = LabelBinarizer().fit(y_test)
    y_test_ = lb.transform(y_test)
    predict_proba = lb.transform(predicted)
    '''-------------'''

    y_test = array(list(y_test))
    predicted = array(list(predicted))
    y_train = array(list(y_train))
    
    # accuracy_train = svm.score(x_train, y_train) #x_train, y_train
    # print(accuracy_train)
#     rocaucscore = roc_auc_score(y_test_, predict_proba, multi_class='ovo',labels=arange(1,7), average='macro')
    rocaucscore = roc_auc_score(y_test_, predict_proba, average='macro')

    
    acc_scikit = accuracy_score(y_test, predicted)
#     accuracy_test = classifier.score(x_test, y_test) #x_test, y_test
    f1score = f1_score(y_test, predicted, average='macro')

    return acc_scikit, rocaucscore, f1score

def train_test_baseline(train_data, test_data):
    temp = train_data.copy()
    del temp['class']
    x_train = temp
    
    temp_test = test_data.copy()
    del temp_test['class']
    x_test = temp_test
    
    y_train = train_data['class']
    y_test = test_data['class']
    
    return x_train, x_test, y_train, y_test

### 以下方式為跑結果，分為統整跑(一次跑全部)跟個別跑(跑單一資料集的單一演算法)

### 1. 單一個別跑請參照以下，用途是作為確認程式碼是否OK時使用
[並請確認有引入單一資料集->點我回上面](#singleRunHead)

In [ ]:
accuracy_container = []
rocauc_container = []
f1score_container = []
acc_scikit_con = []
SUM_OF_COLUMNS = 0
for i in range(5):
    AfterFeatureSelection, len_of_columns = fs_PCA(training_Data[i])
    x_train, x_test, y_train, y_test = train_test_clean(training_Data[i], testing_Data[i], AfterFeatureSelection)
    SUM_OF_COLUMNS = SUM_OF_COLUMNS + len_of_columns
    acc_scikit, rocaucs, f1score = train_model(x_train, x_test, y_train, y_test, classifier)
    # print(accuracy_U)
    acc_scikit_con.append(acc_scikit)
#     accuracy_container.append(accuracy)
    rocauc_container.append(rocaucs)
    f1score_container.append(f1score)
    print(f'accuracy of fold {i+1}: {acc_scikit}')
    print(f'rocaucs of fold {i+1}: {rocaucs}')
avg_acc_scikit = sum(acc_scikit_con)/5
# avg_acc = sum(accuracy_container)/5
avg_rocauc = sum(rocauc_container)/5
avg_f1score = sum(f1score_container)/5
print('--------Baseline2 MI--------')
# print(f'avg_acc: {avg_acc}')
print(f'avg_acc_scikit: {avg_acc_scikit}')
print(f'avg_rocaucscore: {avg_rocauc}')
print(f'avg_f1score: {avg_f1score}')
print(f'avg_len: {SUM_OF_COLUMNS/5}')

### 2. 統整跑需要先做讀寫檔案操作

In [13]:
def writeFiles_Decom_UorI(STORE_PATH, DECOMPOSITION_TYPE, UorI, foldTurn, choosedFeature, f_len,FS_METHOD):
    try:
        file_U = open(STORE_PATH + f'/{DECOMPOSITION_TYPE}/{UorI}/{FS_METHOD}_folds{foldTurn+1}.txt','w')
    except:
        os.makedirs(STORE_PATH + f'/{DECOMPOSITION_TYPE}/{UorI}')
        file_U = open(STORE_PATH + f'/{DECOMPOSITION_TYPE}/{UorI}/{FS_METHOD}_folds{foldTurn+1}.txt','w')
    choosedFeature = sorted(choosedFeature)
    for item in choosedFeature:
        file_U.write(item+"\n")
    file_U.write(str(f_len)+"\n")
    file_U.close()

In [14]:
def readLines_UorI(STORE_PATH, DECOMPOSITION_TYPE, UorI, foldTurn,FS_METHOD):
    try:
        file1 = open(STORE_PATH + f'{DECOMPOSITION_TYPE}/{UorI}/{FS_METHOD}_folds{foldTurn+1}.txt', 'r')
        Lines = file1.readlines()
        choosedFeature = []
        for line in Lines:
            choosedFeature.append(line.strip())
        choosedFeature = choosedFeature[:-1]
        fs_len = Lines[-1]
        file1.close()
        return choosedFeature, fs_len
    except:
        return [], 0

In [15]:
result_ova = pd.DataFrame(columns=['datasetName', 'algoName','acc_u', 'acc_i', 'auc_u', 'auc_i', 'f1_u', 'f1_i', 'len_u', 'len_i','spend_time','reductionRate'])
result_ovo = pd.DataFrame(columns=['datasetName', 'algoName','acc_u', 'acc_i', 'auc_u', 'auc_i', 'f1_u', 'f1_i', 'len_u', 'len_i','spend_time','reductionRate'])

### Embedded 統整跑

In [30]:
classifier = knn
CLASSIFIER_STRING = "KNN"
EMBEDDED_ALGO = ['Lasso', 'Ridge', 'RandomForest', 'XGBoost']

dir_data = '../datasets/'
for qq in range(len(DATASETS)):
    DATASET_CSV_NAME = f'{DATASETS_CSV[qq]}.csv'
    DATASET_NAME = f'{DATASETS[qq]}'
    dir_train_test_data = f'../train_test_datasets/{DATASET_NAME}'

    train_data_loc = os.path.join(dir_data, DATASET_CSV_NAME)
    print('Path of read in data: %s' % (train_data_loc))
    data = pd.read_csv(train_data_loc)
    # data
    CLASS_AMOUNT = data['class'].value_counts().size
    FEATURE_AMOUNT = len(data.columns)-2
    # CLASS_AMOUNT

    train_1_data_loc = os.path.join(dir_train_test_data, 'train_1.csv')
    train_1 = pd.read_csv(train_1_data_loc)
    train_2_data_loc = os.path.join(dir_train_test_data, 'train_2.csv')
    train_2 = pd.read_csv(train_2_data_loc)
    train_3_data_loc = os.path.join(dir_train_test_data, 'train_3.csv')
    train_3 = pd.read_csv(train_3_data_loc)
    train_4_data_loc = os.path.join(dir_train_test_data, 'train_4.csv')
    train_4 = pd.read_csv(train_4_data_loc)
    train_5_data_loc = os.path.join(dir_train_test_data, 'train_5.csv')
    train_5 = pd.read_csv(train_5_data_loc)
    test_1_data_loc = os.path.join(dir_train_test_data, 'test_1.csv')
    test_1 = pd.read_csv(test_1_data_loc)
    test_2_data_loc = os.path.join(dir_train_test_data, 'test_2.csv')
    test_2 = pd.read_csv(test_2_data_loc)
    test_3_data_loc = os.path.join(dir_train_test_data, 'test_3.csv')
    test_3 = pd.read_csv(test_3_data_loc)
    test_4_data_loc = os.path.join(dir_train_test_data, 'test_4.csv')
    test_4 = pd.read_csv(test_4_data_loc)
    test_5_data_loc = os.path.join(dir_train_test_data, 'test_5.csv')
    test_5 = pd.read_csv(test_5_data_loc)
    del train_1['numbers']
    del train_2['numbers']
    del train_3['numbers']
    del train_4['numbers']
    del train_5['numbers']
    del test_1['numbers']
    del test_2['numbers']
    del test_3['numbers']
    del test_4['numbers']
    del test_5['numbers']
    training_Data = []
    training_Data.append(train_1)
    training_Data.append(train_2)
    training_Data.append(train_3)
    training_Data.append(train_4)
    training_Data.append(train_5)
    testing_Data = []
    testing_Data.append(test_1)
    testing_Data.append(test_2)
    testing_Data.append(test_3)
    testing_Data.append(test_4)
    testing_Data.append(test_5)
    
    spend_time_OVA_container = []
    spend_time_OVO_container = []
    
    for z in range(len(EMBEDDED_ALGO)):
        spend_time_OVA = 0
        spend_time_OVO = 0
        STORE_PATH = f'./EMBEDDED_{EMBEDDED_ALGO[z]}/{DATASET_NAME}/'
        print(f'------------------{EMBEDDED_ALGO[z]}-----------------')
        
        for i in range(5):     
            cols_container_OVA = []
            cols_container_OVO = []
            start_time = time.time()
            #ova
            start_time_OVA = time.time()
            data_AfterDecomposition_OVA = decomposition(training_Data[i])
            cols_container_AfterFeatureSelection_OVA = fs_embedded_ova(cols_container_OVA, data_AfterDecomposition_OVA, EMBEDDED_ALGO[z])
            end_time_OVA = time.time()
            spend_time_OVA += end_time_OVA - start_time_OVA
            #ovo
            start_time_OVO = time.time()
            data_AfterDecomposition_OVO, looptimes = decomposition_ovo(training_Data[i])
            cols_container_AfterFeatureSelection_OVO = fs_embedded_ovo(cols_container_OVO, data_AfterDecomposition_OVO, looptimes, EMBEDDED_ALGO[z])
            end_time_OVO = time.time()
            spend_time_OVO += end_time_OVO - start_time_OVO

            # union
            choosedFeature_U_OVA, len_union_OVA = Union(cols_container_AfterFeatureSelection_OVA)
            choosedFeature_U_OVO, len_union_OVO = Union(cols_container_AfterFeatureSelection_OVO)
            
            if(choosedFeature_U_OVA != []):
                writeFiles_Decom_UorI(STORE_PATH, 'OVA', 'U', i, choosedFeature_U_OVA, len_union_OVA, EMBEDDED_ALGO[z])
            if(choosedFeature_U_OVO != []):
                writeFiles_Decom_UorI(STORE_PATH, 'OVO', 'U', i, choosedFeature_U_OVO, len_union_OVO, EMBEDDED_ALGO[z])
                
            # intersection
            choosedFeature_I_OVA, len_intersection_OVA = Intersection(cols_container_AfterFeatureSelection_OVA)
            choosedFeature_I_OVO, len_intersection_OVO = Intersection(cols_container_AfterFeatureSelection_OVO)
            if(choosedFeature_I_OVA != []):
                writeFiles_Decom_UorI(STORE_PATH, 'OVA', 'I', i, choosedFeature_I_OVA, len_intersection_OVA, EMBEDDED_ALGO[z])
            if(choosedFeature_I_OVO != []):
                writeFiles_Decom_UorI(STORE_PATH, 'OVO', 'I', i, choosedFeature_I_OVO, len_intersection_OVO, EMBEDDED_ALGO[z])  
        
        spend_time_OVA = spend_time_OVA/5
        spend_time_OVO = spend_time_OVO/5
        spend_time_OVA_container.append(spend_time_OVA)
        spend_time_OVO_container.append(spend_time_OVO)
            
    TYPES = ['OVA', 'OVO']
    for kk in range(len(EMBEDDED_ALGO)):
        STORE_PATH = f'./EMBEDDED_{EMBEDDED_ALGO[kk]}/{DATASET_NAME}/'
        for k in TYPES:
            accuracy_container_U = []
            accuracy_container_I = []
            rocaucscore_U = []
            rocaucscore_I = []
            f1_scoreContainer_U = []
            f1_scoreContainer_I = []
            SUM_OF_U = 0
            SUM_OF_I = 0
            for i in range(5):
                choosedFeature_AfterUnion_U, len_union = readLines_UorI(STORE_PATH, k, 'U', i, EMBEDDED_ALGO[kk])
                if(choosedFeature_AfterUnion_U != []):
                    SUM_OF_U = SUM_OF_U + int(len_union)
                    x_train_U, x_test_U, y_train_U, y_test_U = train_test_clean(training_Data[i], testing_Data[i], choosedFeature_AfterUnion_U)
                    accuracy_U,  rocaucs_U, f1score_U = train_model(x_train_U, x_test_U, y_train_U, y_test_U, classifier)
                    accuracy_container_U.append(accuracy_U)
                    rocaucscore_U.append(rocaucs_U)
                    f1_scoreContainer_U.append(f1score_U)
                else:
                    accuracy_container_U.append(0)
                    rocaucscore_U.append(0)
                    f1_scoreContainer_U.append(0)

                choosedFeature_AfterUnion_I, len_intersection = readLines_UorI(STORE_PATH, k, 'I', i, EMBEDDED_ALGO[kk])
                if(choosedFeature_AfterUnion_I != []):
                    SUM_OF_I = SUM_OF_I + int(len_intersection)

                    x_train_I, x_test_I, y_train_I, y_test_I = train_test_clean(training_Data[i], testing_Data[i], choosedFeature_AfterUnion_I)
                    accuracy_I, rocaucs_I, f1score_I = train_model(x_train_I, x_test_I, y_train_I, y_test_I, classifier)
                    accuracy_container_I.append(accuracy_I)
                    rocaucscore_I.append(rocaucs_I)
                    f1_scoreContainer_I.append(f1score_I)    
                else:
                    accuracy_container_I.append(0)
                    rocaucscore_I.append(0)
                    f1_scoreContainer_I.append(0)
                    
            avg_acc_U = round(sum(accuracy_container_U)/5,3)
            avg_rocaucscore_U = round(sum(rocaucscore_U)/5,3)
            avg_f1score_U = round(sum(f1_scoreContainer_U)/5,3)

            avg_acc_I = round(sum(accuracy_container_I)/5,3)
            avg_rocaucscore_I = round(sum(rocaucscore_I)/5,3)
            avg_f1score_I = round(sum(f1_scoreContainer_I)/5,3)

            print(f'-----------{EMBEDDED_ALGO[kk]},{k}----------')
            print(f'U:avg_acc: {avg_acc_U}')
            print(f'U:avg_rocaucscore: {avg_rocaucscore_U}')
            print(f'U:avg_f1score: {avg_f1score_U}')
            print(f'U:avg_union_len: {SUM_OF_U/5}')
            print(f'I:avg_acc: {avg_acc_I}')
            print(f'I:avg_rocaucscore: {avg_rocaucscore_I}')
            print(f'I:avg_f1score: {avg_f1score_I}')
            print(f'I:avg_intersection_len: {SUM_OF_I/5}')
            
            if(k == 'OVA'):
                spend_time = spend_time_OVA_container[kk]
                reductionRate = round((SUM_OF_U/5)/FEATURE_AMOUNT, 3)
            elif(k == 'OVO'):
                spend_time = spend_time_OVO_container[kk]
                reductionRate = round((SUM_OF_I/5)/FEATURE_AMOUNT, 3)

            t = pd.DataFrame([{'datasetName':DATASET_NAME,
                         'algoName':EMBEDDED_ALGO[kk],
                         'acc_u':str(avg_acc_U),
                         'acc_i':str(avg_acc_I),
                         'auc_u':str(avg_rocaucscore_U),
                         'auc_i':str(avg_rocaucscore_I),
                         'f1_u':str(avg_f1score_U),
                         'f1_i':str(avg_f1score_I),
                         'len_u':str(math.ceil(SUM_OF_U/5)),
                         'len_i':str(math.ceil(SUM_OF_I/5)),
                         'spend_time':str(spend_time),
                         'reductionRate':str(reductionRate)
                        }])
            if k == 'OVA':
                result_ova = pd.concat([result_ova, t]).reset_index(drop=True).astype('string')
            elif k == 'OVO':
                result_ovo = pd.concat([result_ovo, t]).reset_index(drop=True).astype('string')

    
with pd.ExcelWriter(f'./result_excel_embedded/result_decom_fs_embedded_{CLASSIFIER_STRING}.xlsx') as writer:
    result_ova.to_excel(writer, sheet_name='ova_svm')
    result_ovo.to_excel(writer, sheet_name='ovo_svm') 

Path of read in data: ../datasets/2_Landsat.csv
------------------Lasso-----------------
------------------Ridge-----------------
------------------RandomForest-----------------
------------------XGBoost-----------------
-----------Lasso,OVA----------
U:avg_acc: 0.872
U:avg_rocaucscore: 0.918
U:avg_f1score: 0.847
U:avg_union_len: 33.6
I:avg_acc: 0.0
I:avg_rocaucscore: 0.0
I:avg_f1score: 0.0
I:avg_intersection_len: 0.0
-----------Lasso,OVO----------
U:avg_acc: 0.869
U:avg_rocaucscore: 0.917
U:avg_f1score: 0.844
U:avg_union_len: 33.8
I:avg_acc: 0.0
I:avg_rocaucscore: 0.0
I:avg_f1score: 0.0
I:avg_intersection_len: 0.0
-----------Ridge,OVA----------
U:avg_acc: 0.869
U:avg_rocaucscore: 0.917
U:avg_f1score: 0.844
U:avg_union_len: 34.0
I:avg_acc: 0.869
I:avg_rocaucscore: 0.917
I:avg_f1score: 0.844
I:avg_intersection_len: 34.0
-----------Ridge,OVO----------
U:avg_acc: 0.869
U:avg_rocaucscore: 0.917
U:avg_f1score: 0.844
U:avg_union_len: 34.0
I:avg_acc: 0.687
I:avg_rocaucscore: 0.799
I:avg_f1sco

-----------Ridge,OVO----------
U:avg_acc: 0.463
U:avg_rocaucscore: 0.667
U:avg_f1score: 0.415
U:avg_union_len: 132.2
I:avg_acc: 0.463
I:avg_rocaucscore: 0.667
I:avg_f1score: 0.415
I:avg_intersection_len: 119.0
-----------RandomForest,OVA----------
U:avg_acc: 0.463
U:avg_rocaucscore: 0.667
U:avg_f1score: 0.415
U:avg_union_len: 147.0
I:avg_acc: 0.457
I:avg_rocaucscore: 0.66
I:avg_f1score: 0.401
I:avg_intersection_len: 119.6
-----------RandomForest,OVO----------
U:avg_acc: 0.463
U:avg_rocaucscore: 0.667
U:avg_f1score: 0.415
U:avg_union_len: 127.6
I:avg_acc: 0.731
I:avg_rocaucscore: 0.833
I:avg_f1score: 0.713
I:avg_intersection_len: 6.0
-----------XGBoost,OVA----------
U:avg_acc: 0.463
U:avg_rocaucscore: 0.667
U:avg_f1score: 0.415
U:avg_union_len: 145.6
I:avg_acc: 0.169
I:avg_rocaucscore: 0.252
I:avg_f1score: 0.131
I:avg_intersection_len: 0.8
-----------XGBoost,OVO----------
U:avg_acc: 0.463
U:avg_rocaucscore: 0.667
U:avg_f1score: 0.415
U:avg_union_len: 107.0
I:avg_acc: 0.373
I:avg_rocaucs

-----------XGBoost,OVO----------
U:avg_acc: 0.586
U:avg_rocaucscore: 0.732
U:avg_f1score: 0.54
U:avg_union_len: 309.8
I:avg_acc: 0.034
I:avg_rocaucscore: 0.1
I:avg_f1score: 0.006
I:avg_intersection_len: 0.2
Path of read in data: ../datasets/13_lung.csv
------------------Lasso-----------------
------------------Ridge-----------------
------------------RandomForest-----------------
------------------XGBoost-----------------
-----------Lasso,OVA----------
U:avg_acc: 0.956
U:avg_rocaucscore: 0.906
U:avg_f1score: 0.845
U:avg_union_len: 298.8
I:avg_acc: 0.0
I:avg_rocaucscore: 0.0
I:avg_f1score: 0.0
I:avg_intersection_len: 0.0
-----------Lasso,OVO----------
U:avg_acc: 0.951
U:avg_rocaucscore: 0.931
U:avg_f1score: 0.877
U:avg_union_len: 153.2
I:avg_acc: 0.0
I:avg_rocaucscore: 0.0
I:avg_f1score: 0.0
I:avg_intersection_len: 0.0
-----------Ridge,OVA----------
U:avg_acc: 0.946
U:avg_rocaucscore: 0.93
U:avg_f1score: 0.89
U:avg_union_len: 3312.0
I:avg_acc: 0.946
I:avg_rocaucscore: 0.93
I:avg_f1score